In [ ]:
#! pip install ../input/talibsourceinstall/talib_binary-0.4.19-cp37-cp37m-manylinux1_x86_64.whl
! pip install numpy --upgrade --ignore-installed
! pip install -U tensorflow
! pip install pandas-datareader
! conda install TA-Lib -y
! conda install pandas-ta -y
#! pip install -U git+https://github.com/twopirllc/pandas-ta.git@development
! pip install yfinance
! pip install shap
! pip install boruta
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
! pip install keras
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
! pip install keras-tuner

!pip install -q condacolab
import condacolab
condacolab.install()
! conda install TA-Lib -y
! conda install pandas-ta -y

In [ ]:
import pandas as pd
import numpy as np
import math as math
import scipy as scip
from scipy.stats import norm
import tabulate as table
import shap
import boruta
#tensorflow
import tensorflow

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

import datetime
import os


In [ ]:
# check the tensorflow and keras version
print(f" TensorFlow version is {tensorflow.__version__} \n Keras version is {tensorflow.keras.__version__}")

# Load the TensorBoard notebook extension
%load_ext tensorboard

# tensorflow modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM

from tensorflow.keras.optimizers import Adam, RMSprop 
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [ ]:
def set_seeds(seed=2022): 
    random.seed(seed)
    np.random.seed(seed)
    tensorflow.random.set_seed(seed)

In [ ]:
def writeCSV(df, filename) :
  filepath = r'/content/drive/MyDrive/Colab Notebooks/output data/'+filename+".csv"
  df.to_csv(filepath)
  print("Written CSV : " + filepath)


In [ ]:
# ---------------------------------------------------------------------------------------------------------DATA SOURCING STEP------------------------------------------------------------------------
#strategy
import talib as tlb
import pandas_ta as ta
import yfinance as yf

# download price information of index etfs and plug all the gaps in the data sets
df_SPY = yf.download('SPY', start='2000-01-01', end='2022-08-05', progress=False )[['Open','High','Low', 'Close', 'Adj Close', 'Volume']]
df_DIA = yf.download('DIA', start='2000-01-01', end='2022-08-05', progress=False )[['Open','High','Low', 'Close', 'Adj Close', 'Volume']]
df_QQQ = yf.download('QQQ', start='2000-01-01', end='2022-08-05', progress=False )[['Open','High','Low', 'Close', 'Adj Close', 'Volume']]

df_SPY.ta.strategy("all") # backfill as most starting data points in new features are empty
df_DIA.ta.strategy("all")
df_QQQ.ta.strategy("all")



#download economic indicators and plug all the gaps in the data sets
df_GDP = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/input data/GDP.csv").fillna("ffil")
df_CPI = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/input data/Monthly CPI.csv").fillna("ffil")
df_UNEMP = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/input data//UNRATE.csv").fillna("ffil")
df_30MRTG = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/input data//MORTGAGE30US.csv").fillna("ffil")
df_VIX = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/input data//VIXCLS.csv").fillna("ffil")
df_T3Bills = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/input data//TB3MS.csv").fillna("ffil")
df_10YIR = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/input data//REAINTRATREARAT10Y.csv").fillna("ffil")


#df_SPY.to_csv("/content/drive/MyDrive/Colab Notebooks/output data/df_SPY.csv")
writeCSV(df_SPY, "df_SPY")
writeCSV(df_DIA, "df_DIA")
writeCSV(df_QQQ, "df_QQQ")
#df_DIA.to_csv("/content/drive/MyDrive/Colab Notebooks/output data/df_DIA.csv")
#df_QQQ.to_csv("/content/drive/MyDrive/Colab Notebooks/output data/df_QQQ.csv")

df_SPY = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/output data/df_SPY.csv")
df_DIA = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/output data/df_DIA.csv")
df_QQQ = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/output data/df_QQQ.csv")

# add ecomomic data points as additional features

df_SPY['Date'] = pd.to_datetime(df_SPY['Date'])
df_DIA['Date'] = pd.to_datetime(df_DIA['Date'])
df_QQQ['Date'] = pd.to_datetime(df_QQQ['Date'])

eco_data= [df_GDP, df_CPI, df_UNEMP, df_30MRTG, df_VIX, df_T3Bills, df_10YIR ]

df_SPY_eco = df_SPY
df_DIA_eco = df_DIA
df_QQQ_eco = df_QQQ

for ecodf in eco_data :    
    ecodf['Date'] = pd.to_datetime(ecodf['Date'])
    df_SPY_eco = pd.merge(df_SPY_eco, ecodf, right_on='Date', left_on='Date', how='left')    
    df_DIA_eco = pd.merge(df_DIA_eco, ecodf, right_on='Date', left_on='Date', how='left')
    df_QQQ_eco = pd.merge(df_QQQ_eco, ecodf, right_on='Date', left_on='Date', how='left')


print(df_DIA_eco.shape)
print(df_QQQ_eco.shape)

131it [00:13,  9.83it/s]
131it [00:08, 16.31it/s]
131it [00:07, 16.54it/s]


Written CSV : /content/drive/MyDrive/Colab Notebooks/output data/df_SPY.csv
Written CSV : /content/drive/MyDrive/Colab Notebooks/output data/df_DIA.csv
Written CSV : /content/drive/MyDrive/Colab Notebooks/output data/df_QQQ.csv
(5684, 292)
(5684, 292)


In [ ]:
# # ---------------------------------------------------------------------------------------------------------DATA PRE PROCESSING STEP------------------------------------------------------------------------
    
df_SPY_eco = df_SPY_eco.fillna(method="ffill")
df_DIA_eco = df_DIA_eco.fillna(method="ffill")
df_QQQ_eco = df_QQQ_eco.fillna(method="ffill")

# for missing starting values
df_SPY_eco=df_SPY_eco.fillna(method="bfill")
df_DIA_eco = df_DIA_eco.fillna(method="bfill")
df_QQQ_eco = df_QQQ_eco.fillna(method="bfill")

# for test purposes. All column are completly filled now
writeCSV(df_SPY_eco, "df_SPY_eco")
#df_SPY_eco.to_csv("/content/drive/MyDrive/Colab Notebooks/output data/df_SPY_eco.csv")



Written CSV : /content/drive/MyDrive/Colab Notebooks/output data/df_SPY_eco.csv


In "Python Machine Learning" by Raschka the author provides some guidance on page 111 when to normalize (min-max scale) and when to standardize data:

"Although normalization via min-max scaling is a commonly used technique that is useful when we need values in a bounded interval, standardization can be more practical for many machine learning algorithms. The reason is that many linear models, such as the logistic regression and SVM, [...] initialize the weights to 0 or small random values close to 0. Using standardization, we center the feature columns at mean 0 with standard deviation 1 so that the feature columns take the form of a normal distribution, which makes it easier to learn the weights. Furthermore, standardization maintains useful information about outliers and makes the algorithm less sensitive to them in contrast to min-max scaling, which scales the data to a limited range of values."

Further, Section 6.3 "Preprocessing data" of Scikit-learn documentation states:

"Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data: Gaussian with zero mean and unit variance.

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

For instance, many elements used in the objective function of a learning algorithm (such as the RBF kernel of Support Vector Machines or the l1 and l2 regularizers of linear models) assume that all features are centered around zero and have variance in the same order. If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected."

In [ ]:
# ---------------------------------------------------------------------------------------------------------DATA SCALING STEP------------------------------------------------------------------------
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# scaling all the features using standard scalar to ensure standard normal N(0,1) distribution
# Tranformation or pre-processing
from sklearn.preprocessing import StandardScaler

# we have to exlcude date columns in order to carry out transformation and feature selection. If not, then date will be treated as a feature
fe_df_SPY = df_SPY_eco.loc[:, df_SPY_eco.columns!='Date']
fe_df_DIA = df_DIA_eco.loc[:, df_DIA_eco.columns!='Date']
fe_df_QQQ = df_QQQ_eco.loc[:, df_QQQ_eco.columns!='Date']

# features 
SPY_feature_X = fe_df_SPY.to_numpy()
DIA_feature_X = fe_df_DIA.to_numpy()
QQQ_feature_X = fe_df_QQQ.to_numpy()

print(SPY_feature_X)

gaussian_scaler = StandardScaler()

SPY_feature_Xt = gaussian_scaler.fit_transform(SPY_feature_X) # you have to fit data in before transforming
DIA_feature_Xt = gaussian_scaler.fit_transform(DIA_feature_X) # you have to fit data in before transforming
QQQ_feature_Xt = gaussian_scaler.fit_transform(QQQ_feature_X) # you have to fit data in before transforming


stdTransform_df_SPY = pd.DataFrame(SPY_feature_Xt, columns=fe_df_SPY.keys())
stdTransform_df_DIA = pd.DataFrame(DIA_feature_Xt, columns=fe_df_DIA.keys())
stdTransform_df_QQQ = pd.DataFrame(QQQ_feature_Xt, columns=fe_df_QQQ.keys())

print(stdTransform_df_SPY)

Variance Inflation Factor : 
VIF starts at 1 and has no upper limit.
VIF = 1, no correlation between the independent variable and the other variables
VIF exceeding 5 or 10 indicates high multicollinearity between this independent variable and the others
Multicollinearity can be detected using various methods and one such method is Variable Inflation Factors (VIF).
VIF determines the strength of the correlation between the independent variables. 
It is predicted by taking a variable and regressing it against every other variable.
In order to include as many features as possible for feature selection, we will consider all the features with VIF score < 10. The study of feature selection using VIF indicates that most features selected are stock technical indicators and a few economic indicators (like CPI). Please refer to attached excel spreadsheet for detail. For the most part, SPY, DIA and QQQ have similar features with ViF score under 10 with few variations in that DIA has more features than SPY with ViF score of less than 10. 

In [ ]:
#--start collecting shortlisted deatures
feature_collection_list_SPY = []
feature_collection_list_DIA = []
feature_collection_list_QQQ = []

def feature_collection(shortlistedFeatures, assetName) : 
    collection_list = pd.DataFrame(shortlistedFeatures, columns=[assetName])
    pd.concat([feature_collection_list, collection_list])    
    print("For asset : ", assetName, "...Features count = ", feature_collection_list.shape , "....Feature list after feature engineering: " , feature_collection_list)

In [ ]:
# ---------------------------------------------------------------------------------------------------------START OF FEATURE ENGINEERING------------------------------------------------------------------------

In [ ]:
# hence standard Gaussian scalar will be applied given that most features will align to standard distribution pattern


#------------------------ start of Feature Engineering -----------------------------------------------#
#----- We have about 293 features in all including economic indicators --------#


# Variable Inflation Factor
#VIF starts at 1 and has no upper limit
#VIF = 1, no correlation between the independent variable and the other variables
#VIF exceeding 5 or 10 indicates high multicollinearity between this independent variable and the others
# Multicollinearity can be detected using various methods and one such method is Variable Inflation Factors (VIF).
# VIF determines the strength of the correlation between the independent variables. 
# It is predicted by taking a variable and regressing it against every other variable.

# Import VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Recall we have transformed stock data during PRE-PROCESSING STEP and stored it Xt
SPY_vif = pd.DataFrame()
DIA_vif = pd.DataFrame()
QQQ_vif = pd.DataFrame()

SPY_vif["Features"] = fe_df_SPY.columns # capture feature names
DIA_vif["Features"] = fe_df_DIA.columns # capture feature names
QQQ_vif["Features"] = fe_df_QQQ.columns # capture feature names

SPY_vif["VIF Factor"] = [variance_inflation_factor(SPY_feature_Xt, i) for i in range(SPY_feature_Xt.shape[1])] # We are computing ViF for every of features in tranformed stockdata set Xt
DIA_vif["VIF Factor"] = [variance_inflation_factor(DIA_feature_Xt, i) for i in range(DIA_feature_Xt.shape[1])] # We are computing ViF for every of features in tranformed stockdata set Xt
QQQ_vif["VIF Factor"] = [variance_inflation_factor(QQQ_feature_Xt, i) for i in range(QQQ_feature_Xt.shape[1])] # We are computing ViF for every of features in tranformed stockdata set Xt

# for the considerations of DL we will choose features with VIF score of less than 10
print("SPY Features with VIF Factor of less than 10 : \n" , SPY_vif[SPY_vif['VIF Factor']<10])
print("DIA Features with VIF Factor of less than 10 : \n" , DIA_vif[DIA_vif['VIF Factor']<10])
print("QQQ Features with VIF Factor of less than 10 : \n" , QQQ_vif[QQQ_vif['VIF Factor']<10])

writeCSV(SPY_vif[SPY_vif['VIF Factor']<10], "SPY_VIF_Factor_LessThanTen")
writeCSV(QQQ_vif[QQQ_vif['VIF Factor']<10], "QQQ_VIF_Factor_LessThanTen")
writeCSV(DIA_vif[DIA_vif['VIF Factor']<10], "DIA_VIF_Factor_LessThanTen")

In [ ]:
feature_collection_list_SPY.append(SPY_vif[SPY_vif['VIF Factor']<10]['Features'].values.tolist())
feature_collection_list_DIA.append(DIA_vif[DIA_vif['VIF Factor']<10]['Features'].values.tolist())
feature_collection_list_QQQ.append(QQQ_vif[QQQ_vif['VIF Factor']<10]['Features'].values.tolist())

print(feature_collection_list_SPY)
print(feature_collection_list_DIA)
print(feature_collection_list_QQQ)

[['ADOSC_3_10', 'ADX_14', 'AR_26', 'BR_26', 'CDL_2CROWS', 'CDL_3BLACKCROWS', 'CDL_3INSIDE', 'CDL_3LINESTRIKE', 'CDL_3OUTSIDE', 'CDL_3WHITESOLDIERS', 'CDL_ADVANCEBLOCK', 'CDL_BELTHOLD', 'CDL_CLOSINGMARUBOZU', 'CDL_COUNTERATTACK', 'CDL_DARKCLOUDCOVER', 'CDL_DOJISTAR', 'CDL_ENGULFING', 'CDL_EVENINGDOJISTAR', 'CDL_EVENINGSTAR', 'CDL_GAPSIDESIDEWHITE', 'CDL_GRAVESTONEDOJI', 'CDL_HAMMER', 'CDL_HANGINGMAN', 'CDL_HARAMI', 'CDL_HARAMICROSS', 'CDL_HIGHWAVE', 'CDL_HIKKAKE', 'CDL_HIKKAKEMOD', 'CDL_HOMINGPIGEON', 'CDL_IDENTICAL3CROWS', 'CDL_INNECK', 'CDL_INSIDE', 'CDL_INVERTEDHAMMER', 'CDL_LADDERBOTTOM', 'CDL_LONGLINE', 'CDL_MARUBOZU', 'CDL_MATCHINGLOW', 'CDL_MORNINGDOJISTAR', 'CDL_MORNINGSTAR', 'CDL_ONNECK', 'CDL_PIERCING', 'CDL_RICKSHAWMAN', 'CDL_SEPARATINGLINES', 'CDL_SHOOTINGSTAR', 'CDL_SHORTLINE', 'CDL_SPINNINGTOP', 'CDL_STALLEDPATTERN', 'CDL_STICKSANDWICH', 'CDL_TASUKIGAP', 'CDL_THRUSTING', 'CDL_TRISTAR', 'CDL_UNIQUE3RIVER', 'CDL_UPSIDEGAP2CROWS', 'CDL_XSIDEGAP3METHODS', 'CHOP_14_1_100', 'DPO

In [ ]:
# features 
# LOGRET_1

# = SPY.to_numpy()
print("Features Array Shape, SPY_X : ", SPY_feature_X.shape)
print("Features Array Shape, DIA_X : ", DIA_feature_X.shape)
print("Features Array Shape, QQQ_X : ", QQQ_feature_X.shape)
# Target
#SPY_feature_Yt = np.where(fe_df_SPY['LOGRET_1'] > 0, 1, 0 )
#DIA_feature_Yt = np.where(fe_df_DIA['LOGRET_1'] > 0, 1, 0 )
#QQQ_feature_Yt = np.where(fe_df_QQQ['LOGRET_1'] > 0, 1, 0 )

SPY_feature_Yt = stdTransform_df_SPY['Adj Close'].to_numpy()
DIA_feature_Yt = stdTransform_df_DIA['Adj Close'].to_numpy()
QQQ_feature_Yt = stdTransform_df_QQQ['Adj Close'].to_numpy()

# Splitting the datasets into training and testing data. Using 20pct of data for test sample
# Always keep shuffle = False for financial time series
SPY_feature_X_train, SPY_feature_X_test, SPY_feature_Yt_train, SPY_feature_Yt_test = train_test_split(SPY_feature_Xt, SPY_feature_Yt, test_size=0.2, random_state=0, shuffle=False)
DIA_feature_X_train, DIA_feature_X_test, DIA_feature_Yt_train, DIA_feature_Yt_test = train_test_split(DIA_feature_Xt, DIA_feature_Yt, test_size=0.2, random_state=0, shuffle=False)
QQQ_feature_X_train, QQQ_feature_X_test, QQQ_feature_Yt_train, QQQ_feature_Yt_test = train_test_split(QQQ_feature_Xt, QQQ_feature_Yt, test_size=0.2, random_state=0, shuffle=False)

# Output the train and test data size
print("SPY Train size : ", SPY_feature_X_train.shape, " | SPY Test size : ", SPY_feature_X_test.shape)
print("DIA Train size : ", DIA_feature_X_train.shape, " | DIA Test size : ", DIA_feature_X_test.shape)
print("QQQ Train size : ", QQQ_feature_X_train.shape, " | QQQ Test size : ", QQQ_feature_X_test.shape)

# Output the train and test data size
print("SPY Train size : ", SPY_feature_Yt_train.shape, " | SPY Test size : ", SPY_feature_Yt_test.shape)
print("DIA Train size : ", DIA_feature_Yt_train.shape, " | DIA Test size : ", DIA_feature_Yt_test.shape)
print("QQQ Train size : ", QQQ_feature_Yt_train.shape, " | QQQ Test size : ", QQQ_feature_Yt_test.shape)

Features Array Shape, SPY_X :  (5684, 291)
Features Array Shape, DIA_X :  (5684, 291)
Features Array Shape, QQQ_X :  (5684, 291)
SPY Train size :  (4547, 291)  | SPY Test size :  (1137, 291)
DIA Train size :  (4547, 291)  | DIA Test size :  (1137, 291)
QQQ Train size :  (4547, 291)  | QQQ Test size :  (1137, 291)
SPY Train size :  (4547,)  | SPY Test size :  (1137,)
DIA Train size :  (4547,)  | DIA Test size :  (1137,)
QQQ Train size :  (4547,)  | QQQ Test size :  (1137,)


In [ ]:
# ---------------------------------------------------------------------------------------------------------SELECT K BEST------------------------------------------------------------------------
# select K best
def runSelectKFeatures (f_regression, Xt, Yt, columnValues, assetName) : # Yt is the target. Xt is the transformed stock / asset price data
    
    scoring_method = SelectKBest(f_regression, k=100) # selecting top 100 features
    scoring_method.fit(Xt, Yt)
    i = 0
    topRankingFeaturesList = []
    
    # Show selected features
    topScoringFeatures = scoring_method.get_support(indices=True)    
    df_selectKbestScores = pd.DataFrame(columns=['SelectKBest Score', 'feature_names'])
    df_selectKbestScores['SelectKBest Score'] = scoring_method.scores_
    df_selectKbestScores['feature_names'] = columnValues
       
    print("Top scoring features sequeunce number for Data : ", assetName, " : ")
    for items in topScoringFeatures :
        print(df_selectKbestScores.iloc[items]['feature_names'])
        topRankingFeaturesList.append(df_selectKbestScores.iloc[items]['feature_names'])
        i = i + 1
    return topRankingFeaturesList

In [ ]:
# ---------------------------------------------------------------------------------------------------------RFE FEATURE ENGINEERING-----------------------------------------------------------------------
# Feature Selection using RFE - Ridge, Lasso, LinearRegression

def runRFEFeatureSelection (Xt, Yt, columnValues, assetName) :
    
    rfe_method_lasso = RFE(Lasso(), n_features_to_select=100, step=1) # we will select 100 features
    rfe_method_lasso.fit(Xt,Yt)

    rfe_method_ridge = RFE(Ridge(), n_features_to_select=100, step=1) # we will select 100 features
    rfe_method_ridge.fit(Xt,Yt)

    rfe_method_linear = RFE(LinearRegression(), n_features_to_select=100, step=1) # we will select 4 features
    rfe_method_linear.fit(Xt,Yt)

    df_rfe = pd.DataFrame()
    df_rfe ['feature_names'] = columnValues
    df_rfe ['RFE Selection Lasso'] = pd.DataFrame(rfe_method_lasso.support_)
    df_rfe ['RFE Selection Ridge'] = pd.DataFrame(rfe_method_ridge.support_)
    df_rfe ['RFE Selection Linear Regression'] = pd.DataFrame(rfe_method_linear.support_)
    
    print("RFE features for Data : ", assetName, " : ")
    print(df_rfe)
       
    return df_rfe

In [ ]:
# Feature selection using RFE - Ridge, Lasso, Linear Regression
from sklearn.feature_selection import RFECV, RFE
from sklearn.linear_model import LinearRegression, Lasso, Ridge

SPY_selectRidgeFeaturesList = runRFEFeatureSelection (SPY_feature_Xt, SPY_feature_Yt, fe_df_SPY.columns.values, "SPY")
DIA_selectRidgeFeaturesList = runRFEFeatureSelection (DIA_feature_Xt, DIA_feature_Yt, fe_df_DIA.columns.values, "DIA")
QQQ_selectRidgeFeaturesList = runRFEFeatureSelection (QQQ_feature_Xt, QQQ_feature_Yt, fe_df_QQQ.columns.values, "QQQ")

writeCSV(SPY_selectRidgeFeaturesList, "SPY_selectRidgeFeaturesList")
writeCSV(DIA_selectRidgeFeaturesList, "DIA_selectRidgeFeaturesList")
writeCSV(QQQ_selectRidgeFeaturesList, "QQQ_selectRidgeFeaturesList")

In [ ]:
feature_collection_list_SPY.append(SPY_selectRidgeFeaturesList[(SPY_selectRidgeFeaturesList['RFE Selection Lasso']==True)]['feature_names'].values.tolist())
feature_collection_list_SPY.append(SPY_selectRidgeFeaturesList[(SPY_selectRidgeFeaturesList['RFE Selection Ridge']==True)]['feature_names'].values.tolist())
feature_collection_list_SPY.append(SPY_selectRidgeFeaturesList[(SPY_selectRidgeFeaturesList['RFE Selection Linear Regression']==True)]['feature_names'].values.tolist())


feature_collection_list_DIA.append(DIA_selectRidgeFeaturesList[(DIA_selectRidgeFeaturesList['RFE Selection Lasso']==True)]['feature_names'].values.tolist())
feature_collection_list_DIA.append(DIA_selectRidgeFeaturesList[(DIA_selectRidgeFeaturesList['RFE Selection Ridge']==True)]['feature_names'].values.tolist())
feature_collection_list_DIA.append(DIA_selectRidgeFeaturesList[(DIA_selectRidgeFeaturesList['RFE Selection Linear Regression']==True)]['feature_names'].values.tolist())

feature_collection_list_QQQ.append(QQQ_selectRidgeFeaturesList[(QQQ_selectRidgeFeaturesList['RFE Selection Lasso']==True)]['feature_names'].values.tolist())
feature_collection_list_QQQ.append(QQQ_selectRidgeFeaturesList[(QQQ_selectRidgeFeaturesList['RFE Selection Ridge']==True)]['feature_names'].values.tolist())
feature_collection_list_QQQ.append(QQQ_selectRidgeFeaturesList[(QQQ_selectRidgeFeaturesList['RFE Selection Linear Regression']==True)]['feature_names'].values.tolist())


In [ ]:

# Feature Selection - SELECT K-BEST
from sklearn.feature_selection import f_regression, SelectKBest, SelectPercentile

SPY_selectKFeaturesList = runSelectKFeatures (f_regression, SPY_feature_Xt, SPY_feature_Yt, fe_df_SPY.columns.values, "SPY")
DIA_selectKFeaturesList = runSelectKFeatures (f_regression, DIA_feature_Xt, DIA_feature_Yt, fe_df_DIA.columns.values, "DIA")
QQQ_selectKFeaturesList = runSelectKFeatures (f_regression, QQQ_feature_Xt, QQQ_feature_Yt, fe_df_QQQ.columns.values, "QQQ")

writeCSV(pd.DataFrame(SPY_selectKFeaturesList), "SPY_selectKFeaturesList")
writeCSV(pd.DataFrame(DIA_selectKFeaturesList), "DIA_selectKFeaturesList")
writeCSV(pd.DataFrame(QQQ_selectKFeaturesList), "QQQ_selectKFeaturesList")

In [ ]:
feature_collection_list_SPY.append(SPY_selectKFeaturesList)
feature_collection_list_DIA.append(DIA_selectKFeaturesList)
feature_collection_list_QQQ.append(QQQ_selectKFeaturesList)
print(feature_collection_list_SPY)

[['ADOSC_3_10', 'ADX_14', 'AR_26', 'BR_26', 'CDL_2CROWS', 'CDL_3BLACKCROWS', 'CDL_3INSIDE', 'CDL_3LINESTRIKE', 'CDL_3OUTSIDE', 'CDL_3WHITESOLDIERS', 'CDL_ADVANCEBLOCK', 'CDL_BELTHOLD', 'CDL_CLOSINGMARUBOZU', 'CDL_COUNTERATTACK', 'CDL_DARKCLOUDCOVER', 'CDL_DOJISTAR', 'CDL_ENGULFING', 'CDL_EVENINGDOJISTAR', 'CDL_EVENINGSTAR', 'CDL_GAPSIDESIDEWHITE', 'CDL_GRAVESTONEDOJI', 'CDL_HAMMER', 'CDL_HANGINGMAN', 'CDL_HARAMI', 'CDL_HARAMICROSS', 'CDL_HIGHWAVE', 'CDL_HIKKAKE', 'CDL_HIKKAKEMOD', 'CDL_HOMINGPIGEON', 'CDL_IDENTICAL3CROWS', 'CDL_INNECK', 'CDL_INSIDE', 'CDL_INVERTEDHAMMER', 'CDL_LADDERBOTTOM', 'CDL_LONGLINE', 'CDL_MARUBOZU', 'CDL_MATCHINGLOW', 'CDL_MORNINGDOJISTAR', 'CDL_MORNINGSTAR', 'CDL_ONNECK', 'CDL_PIERCING', 'CDL_RICKSHAWMAN', 'CDL_SEPARATINGLINES', 'CDL_SHOOTINGSTAR', 'CDL_SHORTLINE', 'CDL_SPINNINGTOP', 'CDL_STALLEDPATTERN', 'CDL_STICKSANDWICH', 'CDL_TASUKIGAP', 'CDL_THRUSTING', 'CDL_TRISTAR', 'CDL_UNIQUE3RIVER', 'CDL_UPSIDEGAP2CROWS', 'CDL_XSIDEGAP3METHODS', 'CHOP_14_1_100', 'DPO

In [ ]:
# ---------------------------------------------------------------------------------------------------------BORUTA FEATUE ENGINEERING------------------------------------------------------------------------
# Feature selection using Boruta

def runBorutaFeatureSelection (features, Xt, Yt, noEstimators, assetName) : # pass transformed training set as well as target set
    
    print("BORUTA FOR ASSET : ", assetName, "....Estimators : ", noEstimators)
    
    rfc = RandomForestRegressor(n_estimators=noEstimators, n_jobs=1000, max_depth=10)
    boruta_selector = BorutaPy(rfc, n_estimators=noEstimators)        
    print("Running BORUTA : ", assetName)    
    boruta_selector.fit(Xt, Yt)
   
       
    # number of selected features
    print ('\n Number of selected features:')
    print (boruta_selector.n_features_)
    feature_df = pd.DataFrame(features.tolist(), columns=['features'])
    feature_df['rank']=boruta_selector.ranking_
    feature_df = feature_df.sort_values('rank', ascending=True).reset_index(drop=True)
    print ('\n Top %d features:' % boruta_selector.n_features_)
    print (feature_df.head(boruta_selector.n_features_))
    writeCSV(feature_df, assetName+"_"+"select_borutaFeatureDF")
    #feature_df.to_csv('/content/drive/MyDrive/Colab Notebooks/output data/'+assetName+'_'+'boruta-feature-ranking.csv', index=False)

    # check ranking of features
    print ('\n Feature ranking:')
    print (boruta_selector.ranking_)
    return feature_df.head(boruta_selector.n_features_)


In [ ]:
# Feature selection using Boruta
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

numner_of_Estimators = 200 # this is an optimal number given that there is marginal benefit for the increase in computational power by increasing trees

SPY_boruta_features = runBorutaFeatureSelection (fe_df_SPY.columns,SPY_feature_Xt, SPY_feature_Yt, numner_of_Estimators, "SPY")
writeCSV(SPY_boruta_features, "SPY_boruta_features_FULL")


DIA_boruta_features = runBorutaFeatureSelection (fe_df_DIA.columns,DIA_feature_Xt, DIA_feature_Yt, numner_of_Estimators, "DIA")
writeCSV(DIA_boruta_features, "DIA_boruta_features_FULL")

QQQ_boruta_featues  = runBorutaFeatureSelection (fe_df_QQQ.columns,QQQ_feature_Xt, QQQ_feature_Yt, numner_of_Estimators, "QQQ")
writeCSV(QQQ_boruta_features, "QQQ_boruta_features_FULL")

In [ ]:
#load boruta features

feature_collection_list_SPY.append(SPY_boruta_features['features'].values.tolist())
feature_collection_list_DIA.append(DIA_boruta_features['features'].values.tolist())
feature_collection_list_QQQ.append(QQQ_boruta_features['features'].values.tolist())

In [ ]:
# ---------------------------------------------------------------------------------------------------------DATA PRUNING STEP------------------------------------------------------------------------
# prepare the data cut with shortlisted features for all the asset types
# all the features extracted by methods above have been written to csv. We will prepare the final list of all features for all assets and load it here to create a final data set

final_SPY_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/input data/final_SPY_features.csv", header=None)
final_DIA_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/input data/final_DIA_features.csv", header=None)
final_QQQ_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/input data/final_QQQ_features.csv", header=None)

final_SPY_rawData = pd.DataFrame()
final_DIA_rawData = pd.DataFrame()
final_QQQ_rawData = pd.DataFrame()

print("SPY final features count : ", len(final_SPY_features))
print("DIA final features count : ", len(final_DIA_features))
print("QQQ final features count : ", len(final_QQQ_features))

spy_list = ['PSARl_0.02_0.2','QTL_30_0.5','RMA_10','SINWMA_14','SMA_10','SUPERT_7_3.0','SUPERTl_7_3.0','T3_10_0.7','TEMA_10','TOS_STDEVALL_LR','TOS_STDEVALL_L_1','TOS_STDEVALL_U_1','TOS_STDEVALL_L_2','TOS_STDEVALL_U_2','TOS_STDEVALL_L_3','TOS_STDEVALL_U_3','VWAP_D','VWMA_10','WCP','WMA_10','ZL_EMA_10','High','Low','Close','Adj Close','ACCBL_20','ACCBM_20','ACCBU_20','ALMA_10_6.0_0.85','OBVe_12','CKSPl_10_3_20','DEMA_10','DCM_20_20','DCU_20_20','EMA_10','FWMA_10','HA_high','HA_low','HA_close','HILO_13_21','HL2','HLC3','HWM','ISB_26','IKS_26','KAMA_10_2_30','KCLe_20_2','KCBe_20_2','KCUe_20_2','MCGD_10','MEDIAN_30','MIDPOINT_2','MIDPRICE_2','OHLC4','PVI_1','PVT','PWMA_10','SSF_10_2','SWMA_10','TRIMA_10','VIDYA_14','MORTGAGE30US','REAINTRATREARAT10Y','Open','ABER_ZG_5_15','ABER_SG_5_15','ABER_XG_5_15','AD','OBV','OBV_min_2','OBV_max_2','OBVe_4','BBL_5_2.0','BBM_5_2.0','BBU_5_2.0','LDECAY_5','DCL_20_20','HA_open','HILOl_13_21','HILOs_13_21','HMA_10','HWU','HWL','HWMA_0.2_0.1_0.1','ITS_9','JMA_7_0','LR_14','NVI_1','PSARs_0.02_0.2','QS_10','SLOPE_1','SMI_5_20_5','STCmacd_10_12_26_0.5','STDEV_30','SUPERTs_7_3.0','THERMOma_20_2_0.5','ZS_30','GDP','UNRATE','VIXCLS']
final_SPY_rawData= fe_df_SPY[spy_list] 

DIA_list = ['SUPERTl_7_3.0','T3_10_0.7','TEMA_10','TOS_STDEVALL_U_2','TOS_STDEVALL_L_3','TOS_STDEVALL_U_3','VWAP_D','WCP','ZL_EMA_10','ACCBL_20','ACCBM_20','ALMA_10_6.0_0.85','ABER_ZG_5_15','ABER_SG_5_15','OBV_min_2','BBL_5_2.0','BBM_5_2.0','BBU_5_2.0','QTL_30_0.5','RMA_10','SINWMA_14','TOS_STDEVALL_LR','TOS_STDEVALL_L_1','TOS_STDEVALL_U_1','TOS_STDEVALL_L_2','VWMA_10','WMA_10','High','Low','Close','Adj Close','ACCBU_20','OBVe_12','DEMA_10','DCM_20_20','EMA_10','FWMA_10','HA_high','HA_low','HA_close','HL2','HLC3','IKS_26','KAMA_10_2_30','KCLe_20_2','KCBe_20_2','KCUe_20_2','MCGD_10','MEDIAN_30','MIDPOINT_2','MIDPRICE_2','OHLC4','PVI_1','SSF_10_2','SWMA_10','TRIMA_10','VIDYA_14','MORTGAGE30US','REAINTRATREARAT10Y','ABER_XG_5_15','AD','OBV','OBV_max_2','OBVe_4','LDECAY_5','HMA_10','HWU','HWL','JMA_7_0','NVI_1','ABER_ATR_5_15','ATRr_14','CKSPs_10_3_20','PSARl_0.02_0.2','SMA_10','SUPERT_7_3.0','CKSPl_10_3_20','DCU_20_20','HILO_13_21','HWM','ISB_26','PVT','PWMA_10','Open','DCL_20_20','HA_open','HILOl_13_21','ITS_9','LR_14','PSARs_0.02_0.2','SUPERTs_7_3.0','THERMOma_20_2_0.5','GDP','UNRATE','DMP_14','AR_26','ISA_9','ICS_26','TRIX_30_9','TRIXs_30_9','ADX_14','AMATe_LR_8_21_2','AMATe_SR_8_21_2']
final_DIA_rawData= fe_df_DIA[DIA_list] 

QQQ_list=['T3_10_0.7','TEMA_10','TOS_STDEVALL_U_2','TOS_STDEVALL_L_3','TOS_STDEVALL_U_3','VWAP_D','WCP','ZL_EMA_10','ACCBL_20','ACCBM_20','ALMA_10_6.0_0.85','ABER_ZG_5_15','ABER_SG_5_15','OBV_min_2','BBL_5_2.0','BBM_5_2.0','BBU_5_2.0','TOS_STDEVALL_LR','TOS_STDEVALL_U_1','TOS_STDEVALL_L_2','VWMA_10','WMA_10','Adj Close','ACCBU_20','SWMA_10','TRIMA_10','ABER_XG_5_15','OBVe_4','SUPERT_7_3.0','SUPERTl_7_3.0','RMA_10','SINWMA_14','TOS_STDEVALL_L_1','High','Low','Close','OBVe_12','DEMA_10','EMA_10','FWMA_10','HA_high','HA_low','HA_close','HL2','HLC3','IKS_26','KCLe_20_2','KCBe_20_2','KCUe_20_2','MCGD_10','MEDIAN_30','MIDPOINT_2','MIDPRICE_2','OHLC4','SSF_10_2','VIDYA_14','AD','OBV','LDECAY_5','HWL','JMA_7_0','ABER_ATR_5_15','ATRr_14','SMA_10','CKSPl_10_3_20','HWM','ISB_26','Open','ISA_9','QTL_30_0.5','DCM_20_20','PVI_1','HMA_10','HWU','NVI_1','CKSPs_10_3_20','PSARl_0.02_0.2','DCU_20_20','HILO_13_21','PVT','PWMA_10','DCL_20_20','HA_open','HILOl_13_21','LR_14','SUPERTs_7_3.0','GDP','DMP_14','HWMA_0.2_0.1_0.1','MEDCPIM158SFRBCLE','TB3MS','ADOSC_3_10','AO_5_34','AOBV_LR_2','AOBV_SR_2']
final_QQQ_rawData= fe_df_QQQ[QQQ_list] 


print("SPY final raw data shape for deep learning : ", final_SPY_rawData.shape)
print("DIA final raw data shape for deep learning : ", final_DIA_rawData.shape)
print("QQQ final raw data shape for deep learning : ", final_QQQ_rawData.shape)



writeCSV(final_SPY_rawData, "SPY_feature_selected_final_rawData")
writeCSV(final_DIA_rawData, "DIA_feature_selected_final_rawData")
writeCSV(final_QQQ_rawData, "QQQ_feature_selected_final_rawData")


print(final_SPY_rawData)


In [ ]:
#--------------------------------------------------------------------------------------DEEP LEARNING --------------------------------------------------------------------------------------------------------------------------#

In [ ]:
# ---------------------------------------------------------------------------------------------------------DATA SCALE AND SPLIT FOR DEEP LEARNING------------------------------------------------------------------------
# define function to scale, split data into training set and test set

def scaleAndTrainTestSplit(data_X, data_Y, scaler, assetName) :
  
  # we require different scalers for X and Y because of the dimensionality. data_x is the training data set with multiple features, data_y is the target training data set with one feature
  if(scaler=='StandardScaler'):
    scaler_x = StandardScaler()
    scaler_y = StandardScaler()

  if(scaler=='MinMax'):
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

  feature_X_train, feature_X_test = train_test_split(data_X, test_size=0.2, random_state=0, shuffle=False)
  feature_Y_train, feature_Y_test = train_test_split(data_Y, test_size=0.2, random_state=0, shuffle=False)

  scaled_feature_X_train = scaler_x.fit_transform(feature_X_train)
  scaled_feature_X_test = scaler_x.transform(feature_X_test)

  scaled_feature_Y_train = scaler_y.fit_transform(feature_Y_train)
  scaled_feature_Y_test = scaler_y.transform(feature_Y_test)

  print("X-train shape: ", scaled_feature_X_train.shape)
  print("X-test  shape: ", scaled_feature_X_test.shape)

  print("Y-train shape: ", scaled_feature_Y_train.shape)
  print("Y-test  shape: ", scaled_feature_Y_test.shape)

  return scaled_feature_X_train, scaled_feature_X_test, scaled_feature_Y_train, scaled_feature_Y_test, scaler_x, scaler_y

In [ ]:
# ---------------------------------------------------------------------------------------------------------DATA SEQUENCING------------------------------------------------------------------------
# sequence split for Deep Learning 
def generate_sequence(data_X, data_Y, sequence_length):
    
    # create X & y data array
    X = []
    Y = []
    
    for i in range (sequence_length, len(data_X)) :
        X.append(data_X[i- sequence_length : i])
        Y.append(data_Y[i])    
      
    return np.array(X), np.array(Y)

In [ ]:
# ---------------------------------------------------------------------------------------------------------STACKED LSTM MODEL COMPILATION------------------------------------------------------------------------
def create_model_stackedLSTM(hu, lookback,ipShape_features):
    print("hu value :", hu, " lookback : ", lookback)
    tensorflow.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()
    model.add(LSTM(units=hu, input_shape=(lookback, ipShape_features), activation = 'elu', return_sequences=True, name='LSTM1'))
    model.add(Dropout(0.25)) # hypertuned dropoutrate
    model.add(LSTM(units=hu, activation = 'relu', return_sequences=True, name='LSTM2'))
    model.add(Dropout(0.25))
    model.add(LSTM(units=hu, activation = 'relu', return_sequences=False, name='LSTM3'))    
    model.add(Dense(units=1, name='Output'))              # can also specify linear activation function 
    
    # specify optimizer separately (preferred method))
    #opt = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    opt = Adam(lr=0.001, epsilon=1e-08, decay=0.0)       # adam optimizer seems to perform better for a single lstm
    
    # model compilation
    model.compile(optimizer=opt, loss='mse', metrics=['mae'])
    
    return model

In [ ]:
# ---------------------------------------------------------------------------------------------------------SINGLE LSTM MODEL COMPILATION------------------------------------------------------------------------
def create_model_singleLSTM(hu, lookback, ipShape_features):
    print("hu value :", hu, " lookback : ", lookback)
    tensorflow.keras.backend.clear_session()   
    
    # instantiate the model
    model = Sequential()
    model.add(LSTM(units=hu, input_shape=(lookback, ipShape_features), activation = 'relu', return_sequences=False, name='LSTM'))
    model.add(Dense(units=1, name='Output'))              # can also specify linear activation function 
    
    # specify optimizer separately (preferred method))
    #opt = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    opt = Adam(lr=0.001, epsilon=1e-08, decay=0.0)       # adam optimizer seems to perform better for a single lstm
    
    # model compilation
    model.compile(optimizer=opt, loss='mse', metrics=['mae'])
    
    return model

In [ ]:
# ---------------------------------------------------------------------------------------------------------MODEL EXECUTION------------------------------------------------------------------------
def callModel(scaled_X_train, scaled_Y_train, scaled_X_test, scaled_Y_test, scaler_y, modelType, hu, lookback, ipShape_features, patience, epochs, batchsize, assetName) :

  print("Asset Type: ", assetName, "...Calling model: ", modelType)
  
  X_train, Y_train = generate_sequence(scaled_X_train, scaled_Y_train, sequence_length=lookback)
  X_test, Y_test = generate_sequence(scaled_X_test, scaled_Y_test, sequence_length=lookback)

  if(modelType=='singleLSTM'):
    model = create_model_singleLSTM(hu, lookback, ipShape_features)
  
  if(modelType=='stackedLSTM'):
    model = create_model_stackedLSTM(hu, lookback,ipShape_features)
  
  # summary
  model.summary()

  model_path = (assetName+'_'+modelType+'_'+'hu_'+str(hu)+'_'+'model.h5')
  logdir = os.path.join(model_path+"_"+"logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  %tensorboard --logdir logs

  my_callbacks = [
    EarlyStopping(patience=patience, monitor='loss', mode='min', verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath=model_path, verbose=1, monitor='loss', save_best_only=True),
    TensorBoard(log_dir=logdir, histogram_freq=1)
  ]

  # Model fitting
  lstm_training = model.fit(X_train, 
                            Y_train, 
                            batch_size=batchsize, 
                            epochs=epochs, 
                            verbose=0, 
                            callbacks=my_callbacks, 
                            shuffle=False)
  print(logdir)
  

  # calculate rmse of loss function
  train_rmse_scaled = np.sqrt(model.evaluate(X_train, Y_train, verbose=0))
  test_rmse_scaled = np.sqrt(model.evaluate(X_test, Y_test, verbose=0))
  print(f'Train RMSE: {train_rmse_scaled[0]:.4f} | Test RMSE: {test_rmse_scaled[0]:.4f}')

  # predictions
  Y_pred = model.predict(X_test)
  print("Y prediction shape : ", Y_pred.shape)

  print("Comparison between actual and predicted values for asset ", assetName, " : ")

  df_compare = pd.DataFrame({
    'actual': scaler_y.inverse_transform(Y_test).flatten(),
    'prediction': scaler_y.inverse_transform(Y_pred).flatten()})
    #index = y_test[lookback:].index)

  df_compare['spread'] = df_compare['prediction'] - df_compare['actual']
  df_compare['spread % '] = (df_compare['prediction'] - df_compare['actual'])/(df_compare['actual'])

  print(df_compare)

  df_compare.to_csv("/content/drive/MyDrive/Colab Notebooks/output data/"+modelType+"_"+assetName+"_results.csv")


In [ ]:
# ------------------------------------------------------------------------------------------START DEEP LEARNING EXECUTION FOR ALL ASSET CLASSES------------------------------------------------------------------------
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import plot_confusion_matrix, auc, roc_curve, plot_roc_curve

lookback = 60
hu = 30
patience = 20
epochs = 500
batchsize = 64 # multiples of 8

# features - convert to numpty
SPY_feature_X = final_SPY_rawData.to_numpy()
DIA_feature_X = final_DIA_rawData.to_numpy()
QQQ_feature_X = final_QQQ_rawData.to_numpy()

# these are Y values  - target is to train model to guess price. 
SPY_adjClose_Y = fe_df_SPY['Adj Close'].to_numpy()
SPY_adjClose_Y = np.reshape(SPY_adjClose_Y, (-1,1))
DIA_adjClose_Y = fe_df_DIA['Adj Close'].to_numpy()
DIA_adjClose_Y = np.reshape(DIA_adjClose_Y, (-1,1))
QQQ_adjClose_Y = fe_df_QQQ['Adj Close'].to_numpy()
QQQ_adjClose_Y = np.reshape(QQQ_adjClose_Y, (-1,1))

# executing single LSTM model for all three asset type - SPY, DIA, QQQ using StandardScaler or Gaussian Scaling
#scaleAndTrainTestSplit(data_X, data_Y, scaler, assetName) 
#callModel(scaled_X_train, scaled_Y_train, scaled_X_test, scaled_Y_test, scaler_y, modelType, hu, lookback, ipShape_features, patience, epochs, batchsize, assetName)

ipShape_features = final_SPY_rawData.shape[1]
print(ipShape_features)
scaled_feature_X_train, scaled_feature_X_test, scaled_feature_Y_train, scaled_feature_Y_test, scaler_x, scaler_y = scaleAndTrainTestSplit(SPY_feature_X, SPY_adjClose_Y,"StandardScaler", "SPY")
callModel(scaled_feature_X_train, scaled_feature_Y_train, scaled_feature_X_test, scaled_feature_Y_test, scaler_y, 'singleLSTM', hu, lookback, ipShape_features, patience, epochs, batchsize, "SPY")

ipShape_features = final_DIA_rawData.shape[1]
scaled_feature_X_train, scaled_feature_X_test, scaled_feature_Y_train, scaled_feature_Y_test, scaler_x, scaler_y = scaleAndTrainTestSplit(DIA_feature_X, DIA_adjClose_Y,"StandardScaler", "DIA")
callModel(scaled_feature_X_train, scaled_feature_Y_train, scaled_feature_X_test, scaled_feature_Y_test, scaler_y, 'singleLSTM', hu, lookback, ipShape_features, patience, epochs, batchsize, "DIA")

ipShape_features = final_QQQ_rawData.shape[1]
scaled_feature_X_train, scaled_feature_X_test, scaled_feature_Y_train, scaled_feature_Y_test, scaler_x, scaler_y = scaleAndTrainTestSplit(QQQ_feature_X, QQQ_adjClose_Y,'StandardScaler', "QQQ")
callModel(scaled_feature_X_train, scaled_feature_Y_train, scaled_feature_X_test, scaled_feature_Y_test, scaler_y, 'singleLSTM', hu, lookback, ipShape_features, patience, epochs, batchsize, "QQQ")

# executing stacked LSTM model for all three asset type - SPY, DIA, QQQ using StandardScaler or Gaussian Scaling

ipShape_features = final_SPY_rawData.shape[1]
scaled_feature_X_train, scaled_feature_X_test, scaled_feature_Y_train, scaled_feature_Y_test, scaler_x, scaler_y = scaleAndTrainTestSplit(SPY_feature_X, SPY_adjClose_Y,'StandardScaler', "SPY")
callModel(scaled_feature_X_train, scaled_feature_Y_train, scaled_feature_X_test, scaled_feature_Y_test, scaler_y, 'stackedLSTM', hu, lookback,ipShape_features, patience, epochs, batchsize, "SPY")

ipShape_features = final_DIA_rawData.shape[1]
scaled_feature_X_train, scaled_feature_X_test, scaled_feature_Y_train, scaled_feature_Y_test, scaler_x, scaler_y = scaleAndTrainTestSplit(DIA_feature_X, DIA_adjClose_Y,'StandardScaler', "DIA")
callModel(scaled_feature_X_train, scaled_feature_Y_train, scaled_feature_X_test, scaled_feature_Y_test, scaler_y, 'stackedLSTM', hu, lookback,ipShape_features, patience, epochs, batchsize,  "DIA")

ipShape_features = final_QQQ_rawData.shape[1]
scaled_feature_X_train, scaled_feature_X_test, scaled_feature_Y_train, scaled_feature_Y_test, scaler_x, scaler_y = scaleAndTrainTestSplit(QQQ_feature_X, QQQ_adjClose_Y,'StandardScaler', "QQQ")
callModel(scaled_feature_X_train, scaled_feature_Y_train, scaled_feature_X_test, scaled_feature_Y_test, scaler_y, 'stackedLSTM', hu, lookback,ipShape_features, patience, epochs, batchsize, "QQQ")




In [ ]:
# ---------------------------------------------------------------------------------------------------------SINGLE LSTM HYPER TUNE FUNCTION------------------------------------------------------------------------
def hypertune_create_model_singleLSTM(hp):
    print("hypertuning single LSTM model")
    
    tensorflow.keras.backend.clear_session()   
    
    # Tune the number of units in the layers from 2 to 32 to obtain best value of hu
    hu_units = hp.Int('units1', min_value=2, max_value=32, step=4)

    # Tune the activation method to use
    hp_activation = hp.Choice(name = 'activation', values = ['relu', 'elu'], ordered = False)    

    # instantiate the model
    model = Sequential()
    #we will choose input shape of 60 by 100 because we are sticking 60 day lookback period and about 100 features for stock price prediction
    model.add(LSTM(units=hu_units, input_shape=(60, 103), activation = hp_activation, return_sequences=False, name='LSTM'))
    model.add(Dense(units=1, name='Output'))              # can also specify linear activation function 
    
    # specify optimizer separately (preferred method))
    #opt = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    opt = Adam(lr=0.001, epsilon=1e-08, decay=0.0)       # adam optimizer seems to perform better for a single lstm
    
    # model compilation
    model.compile(optimizer=opt, loss='mse', metrics=['mae', 'accuracy'])
    
    return model

In [ ]:
# ---------------------------------------------------------------------------------------------------------STACKED LSTM HYPER TUNE FUNCTION------------------------------------------------------------------------
def hypertune_create_model_stackedLSTM(hp):
    print("hypertuning STACKED LSTM model")
    tensorflow.keras.backend.clear_session()   
    
    # Tune the number of units in the layers from 2 to 32 to obtain best value of hu
    hu_units1 = hp.Int('units1', min_value=2, max_value=32, step=4)
    hu_units2 = hp.Int('units1', min_value=2, max_value=32, step=4)
    hu_units3 = hp.Int('units1', min_value=2, max_value=32, step=4)
    # Tune the activation method to use
    hp_activation1 = hp.Choice(name = 'activation', values = ['relu', 'elu'], ordered = False)    
    hp_activation2 = hp.Choice(name = 'activation', values = ['relu', 'elu'], ordered = False)    
    hp_activation3 = hp.Choice(name = 'activation', values = ['relu', 'elu'], ordered = False)    
    #Tune dropout rates
    hp_dropout1 = hp.Float('Dropout_rate', min_value=0, max_value=0.5, step=0.1)
    hp_dropout2 = hp.Float('Dropout_rate', min_value=0, max_value=0.5, step=0.1)


    # instantiate the model
    model = Sequential()
    model.add(LSTM(units=hu_units1, input_shape=(60, 103), activation = hp_activation1, return_sequences=True, name='LSTM1'))
    model.add(Dropout(hp_dropout1, name='Drouput1')) # 25% dropoutrate
    model.add(LSTM(units=hu_units2, activation = hp_activation2, return_sequences=True, name='LSTM2'))
    model.add(Dropout(hp_dropout2, name='Drouput2'))
    model.add(LSTM(units=hu_units3, activation = hp_activation3, return_sequences=False, name='LSTM3'))    
    model.add(Dense(units=1, name='Output'))              # can also specify linear activation function 
    
    # specify optimizer separately (preferred method))
    #opt = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    opt = Adam(lr=0.001, epsilon=1e-08, decay=0.0)       # adam optimizer seems to perform better for a single lstm
    
    # model compilation
    model.compile(optimizer=opt, loss='mse', metrics=['mae', 'accuracy'])
    
    return model

In [ ]:
!pip install keras-tuner

In [ ]:
# ---------------------------------------------------------------------------------------------------------START HYPER TUNING------------------------------------------------------------------------

In [ ]:
# HYPER TUNE SINGLE LSTM
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner

hp = keras_tuner.HyperParameters()

# initialize an early stopping callback to prevent the model from
# overfitting/spending too much time training with minimal gains
callback1 = [EarlyStopping(patience=20, monitor='loss', mode='min', verbose=1, restore_best_weights=True),
             TensorBoard(log_dir="./tensorboard/rslogs")]

%tensorboard --logdir ./tensorboard/rslogs

scaled_feature_X_train, scaled_feature_X_test, scaled_feature_Y_train, scaled_feature_Y_test, scaler_x, scaler_y = scaleAndTrainTestSplit(DIA_feature_X, DIA_adjClose_Y,"StandardScaler", "SPY")
X_train, Y_train = generate_sequence(scaled_feature_X_train, scaled_feature_Y_train, sequence_length=lookback)
X_test, Y_test = generate_sequence(scaled_feature_X_test, scaled_feature_Y_train, sequence_length=lookback)

# RandomSearch algorithm from keras tuner
rstuner = keras_tuner.RandomSearch(
    hypertune_create_model_singleLSTM,
    objective="val_accuracy",
    max_trials=5,
    directory="./keras",
    project_name="rstrail",
    overwrite=True
)
# launch tuning process
rstuner.search(X_train, Y_train, epochs=50, validation_data=(X_test, Y_test), callbacks=callback1, shuffle=False )
#rstuner.search(g, epochs=50, validation_data=g_, callbacks=callback1, class_weight = class_weight, shuffle=False)
# display the best hyperparameter values for the model based on the defined objective function
best_rshp = rstuner.get_best_hyperparameters()[0]
print(best_rshp.values)
# display tuning results summary
rstuner.results_summary()

In [ ]:
# HYPER TUNE STACKED LSTM

# HYPER TUNE SINGLE LSTM
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner

hp = keras_tuner.HyperParameters()

# initialize an early stopping callback to prevent the model from
# overfitting/spending too much time training with minimal gains
callback1 = [EarlyStopping(patience=20, monitor='loss', mode='min', verbose=1, restore_best_weights=True),
             TensorBoard(log_dir="./tensorboard/rslogs")]

%tensorboard --logdir ./tensorboard/rslogs

scaled_feature_X_train, scaled_feature_X_test, scaled_feature_Y_train, scaled_feature_Y_test, scaler_x, scaler_y = scaleAndTrainTestSplit(DIA_feature_X, DIA_adjClose_Y,"StandardScaler", "DIA")
X_train, Y_train = generate_sequence(scaled_feature_X_train, scaled_feature_Y_train, sequence_length=lookback)
X_test, Y_test = generate_sequence(scaled_feature_X_test, scaled_feature_Y_train, sequence_length=lookback)

# RandomSearch algorithm from keras tuner
rstuner = keras_tuner.RandomSearch(
    hypertune_create_model_stackedLSTM,
    objective="val_accuracy",
    max_trials=5,
    directory="./keras",
    project_name="rstrail",
    overwrite=True
)
# launch tuning process
rstuner.search(X_train, Y_train, epochs=50, validation_data=(X_test, Y_test), callbacks=callback1, shuffle=False )
#rstuner.search(g, epochs=50, validation_data=g_, callbacks=callback1, class_weight = class_weight, shuffle=False)
# display the best hyperparameter values for the model based on the defined objective function
best_rshp = rstuner.get_best_hyperparameters()[0]
print(best_rshp.values)
# display tuning results summary
rstuner.results_summary()

Trial 4 Complete [00h 35m 00s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.0
Total elapsed time: 02h 02m 05s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
18                |22                |units1
relu              |relu              |activation
0.2               |0.5               |Dropout_rate

hypertuning STACKED LSTM model
Epoch 1/50
141/141 [==============================] - 44s 293ms/step - loss: 26.8527 - mae: 1.0074 - accuracy: 0.0000e+00 - val_loss: 567.3817 - val_mae: 9.8701 - val_accuracy: 0.0000e+00
Epoch 2/50
141/141 [==============================] - 43s 308ms/step - loss: 0.4604 - mae: 0.4820 - accuracy: 0.0000e+00 - val_loss: 30594.5762 - val_mae: 56.4444 - val_accuracy: 0.0000e+00
Epoch 3/50
141/141 [==============================] - 41s 287ms/step - loss: 0.2816 - mae: 0.4521 - accuracy: 0.0000e+00 - val_loss: 9875.3516 - val_mae: 23.4557 - val_accuracy: 0.0000e+00
Epoch 4/50
141/141 [==============================] - 40s 286ms/